In [ ]:
!pip install datasets peft bitsandbytes streamlit pyngrok konlpy
!pip install -U bitsandbytes

import os
import torch
import pandas as pd

import nltk
from konlpy.tag import Okt
import re

nltk.download('stopwords')
nltk.download('punkt')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 100.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from nltk.corpus import stopwords
english_stopwords = set(stopwords.words('english'))
okt = Okt()

In [ ]:
def clean_text_columns(df, columns):
    if isinstance(columns, str):
        columns = [columns]

    for col in columns:
        # 전처리: 숫자, 공백, 특수문자 제거 강화
        df[col + '_cleaned'] = (
            df[col]
            .astype(str)
            .str.replace(r"\d+", "", regex=True)  # 숫자 제거
            .str.replace("\t|\n", " ")  # 탭, 줄바꿈 제거
            .str.replace(r" {2,}", " ", regex=True)  # 여러 공백 제거
            .str.replace(r"[\*\-,|!\?\"\'\(\)\[\]{}<>]", "", regex=True)  # 특수문자 제거 (확장)
            .str.strip()  # 앞뒤 공백 제거
        )

        # 불필요한 단어 제거: 한글과 영어 처리 강화
        def remove_unnecessary_words(text):
            if re.search('[가-힣]', text):  # 한국어가 포함된 경우
                tagged = okt.pos(text)  # 품사 태깅
                # 조사(Josa), 접속사(Conjunction), 형용사(Adjective), 부사(Adverb) 제거
                cleaned_words = [
                    word for word, tag in tagged
                    if tag not in ['Josa', 'Conjunction', 'Adjective', 'Adverb']
                ]
                return ' '.join(cleaned_words)
            else:  # 영어인 경우
                words = nltk.word_tokenize(text)  # 단어 토큰화
                # 영어에서 불용어와 짧은 단어 제거 (길이가 2 이하인 단어는 불필요할 수 있음)
                cleaned_words = [
                    word for word in words
                    if word.lower() not in english_stopwords and len(word) > 2
                ]
                return ' '.join(cleaned_words)

        # 'example_cleaned' 열에 대해 전처리 적용
        df[col + '_cleaned'] = df[col + '_cleaned'].apply(remove_unnecessary_words)

        # 빈값을 빈 문자열로 채움
        df[col + '_cleaned'].fillna("", inplace=True)

    return df

In [ ]:
df = pd.read_csv("/content/sorted.csv")
df = clean_text_columns(df, 'example')

<ipython-input-3-4c78370a6426>:40: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col + '_cleaned'].fillna("", inplace=True)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF 벡터화
vectorizer = TfidfVectorizer(max_features=10, stop_words='english')
tfidf_matrix = vectorizer.fit_transform(df['example_cleaned'])

# 각 문서에 대해 핵심어 추출
df['keywords'] = [list(vectorizer.get_feature_names_out()[tfidf_matrix[i].nonzero()[1]]) for i in range(tfidf_matrix.shape[0])]

In [ ]:
!pip install rake-nltk

from rake_nltk import Rake

# RAKE 객체 생성
r = Rake()

# 각 문서에 대해 핵심어 추출
def extract_keywords_rake(text):
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()  # 중요도 순으로 핵심어 리스트 반환

df['keywords_1'] = df['example_cleaned'].apply(extract_keywords_rake)

In [ ]:
pip install keybert

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.3 MB/s eta 0:00:00


In [ ]:
from keybert import KeyBERT

# KeyBERT 객체 생성
kw_model = KeyBERT()

# 각 문서에 대해 핵심어 추출
df['keywords_2'] = df['example_cleaned'].apply(lambda x: kw_model.extract_keywords(x, top_n=5))

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# from gensim.summarization import keywords

# # TextRank를 이용한 핵심어 추출
# def extract_keywords_textrank(text):
#     return keywords(text, words=5, lemmatize=True).split('\n')

# df['keywords_2'] = df['example_cleaned'].apply(extract_keywords_textrank)

In [ ]:
df

,ESRS,GRI,example,기아,toyota,example_cleaned,keywords,keywords_1,keywords_2
0,E1-1,na,기후 관련 전환 계획\n탄소중립 추진 현대자동차는 지속가능한 지구환경을 미래...,NaN,NaN,기후 관련 전환 계획 \n 탄소 중립 추진 현 대 자동차 지속 지 구 환경 미래세 ...,"[자동차, 환경, 위해, 에너지, 사업, 하여, 협력, 관리]",[수 소 연료 전지 상용차 대한 연구개발 투자 지속 년 ‘ 유니버스 수 소 전 기 ...,"[(친환경, 0.3795), (저장장치, 0.3482), (기후변화, 0.3124)..."
1,E1-2,na,전략 및 의사결정 기후변화 완화 및 적응을 위한 노력,NaN,NaN,전략 및 의사결정 기후변화 완화 및 적응 위 노력,[],[전략 및 의사결정 기후변화 완화 및 적응 위 노력],"[(의사결정, 0.6939), (기후변화, 0.5168), (적응, 0.446), ..."
2,E1-3,201-2,기후 관련 전환 계획\n탄소중립 추진 현대자동차는 지속가능한 지구환경을 미래...,NaN,NaN,기후 관련 전환 계획 \n 탄소 중립 추진 현 대 자동차 지속 지 구 환경 미래세 ...,"[자동차, 환경, 위해, 에너지, 사업, 하여, 협력, 관리]",[수 소 연료 전지 상용차 대한 연구개발 투자 지속 년 ‘ 유니버스 수 소 전 기 ...,"[(친환경, 0.3795), (저장장치, 0.3482), (기후변화, 0.3124)..."
3,E1-3,305-5,"자본 배치 전동화 전환을 위해 2032년까지 EV 공장 신설, 배터리 JV,...",NaN,NaN,자본 배치 전 동화 전환 위해 년 EV 공장 신설 배터리 JV 소재 확보 등 전략 ...,"[자동차, 위해, 에너지, 사업, 하여, 협력, 관리, 직원]",[조 원 협력 사 및 타 사 전략 적 제휴 지 분 인수 등 위 전략 투자 사업 장 ...,"[(성과, 0.3544), (전환, 0.3507), (재생에너지, 0.3408), ..."
4,E1-4,305-1,"RE100 이행 계획 현대자동차는 주요 그룹사인 기아, 현대모비스, 현대위아...",NaN,NaN,RE 이행 계획 현 대 자동차 주요 그룹 사인 기아 현대 모비스 현 대위 와 년 전...,"[자동차, 환경, 위해, 에너지, 사업, 하여, 협력, 관리]",[이를 위해 현 대 자동차 국가 별 재생에너지 수급 환경 정부 정책 및 규제 공장 ...,"[(저장장치, 0.399), (친환경, 0.3948), (상용차, 0.3889), ..."
...,...,...,...,...,...,...,...,...,...
615,G1-3,205-1,NaN,NaN,Prevention and detection of\ncorruption and br...,nan,[],[nan],"[(nan, 1.0)]"
616,G1-3,205-2,NaN,NaN,Prevention and detection of\ncorruption and br...,nan,[],[nan],"[(nan, 1.0)]"
617,G1-4,205-3,NaN,NaN,Confirmed incidents of\ncorruption or bribery\...,nan,[],[nan],"[(nan, 1.0)]"
618,G1-5,415-1,NaN,NaN,Political influence and lobbying\nactivities\n...,nan,[],[nan],"[(nan, 1.0)]"


In [ ]:
df = df.drop(['example', 'example_cleaned', 'keywords'], axis = 1)

In [ ]:
df

,ESRS,GRI,keywords_1
0,E1-1,na,[수 소 연료 전지 상용차 대한 연구개발 투자 지속 년 ‘ 유니버스 수 소 전 기 ...
1,E1-2,na,[전략 및 의사결정 기후변화 완화 및 적응 위 노력]
2,E1-3,201-2,[수 소 연료 전지 상용차 대한 연구개발 투자 지속 년 ‘ 유니버스 수 소 전 기 ...
3,E1-3,305-5,[조 원 협력 사 및 타 사 전략 적 제휴 지 분 인수 등 위 전략 투자 사업 장 ...
4,E1-4,305-1,[이를 위해 현 대 자동차 국가 별 재생에너지 수급 환경 정부 정책 및 규제 공장 ...
...,...,...,...
615,G1-3,205-1,[prevention detection corruption bribery compa...
616,G1-3,205-2,[prevention detection corruption bribery compa...
617,G1-4,205-3,[confirmed incidents corruption bribery curren...
618,G1-5,415-1,[political influence lobbying activities toyot...


In [ ]:
df.to_csv('sorted_cleaned.csv', index=False)